# Using Python to apply WCS to Photometry Images

    Created: Zachary Lane, June 2024
    Last Edited: C.Worley, July 2025

This tutorial introduces students using applying the World Coordinate System (WCS) to photometry images. This is an advanced module that is optional and is more difficult to work through.

Work your way through the tutorial, **reading each code cell carefully before running it**. This is a live document, all the cells can easily be edited, and re-run any time you wish. Ask questions if you need to.

In [ ]:
import os # A module for communicating with the operating system e.g. commands and files
import glob # A module for searching data files
import matplotlib.pyplot as plt # A plotting library
import numpy as np # Numerical Python, great for vectorised equations
from scipy.signal import fftconvolve, convolve2d # Scientific Python, great for algorithms and optimisation
import pandas as pd # DataFrames for organising data/tables
from astropy.wcs import WCS # WCS for Python
from astropy.io import fits # Astronomy Python for opening ".fits" files
from astropy.coordinates import angular_separation # A module for calculating angular separations
from photutils.background import Background2D, MedianBackground # Fitting background surfaces to astronomical images
from copy import deepcopy # A module for copying objects
import warnings # To ignore our problems
warnings.filterwarnings('ignore', category=RuntimeWarning) #Shhhhh, ignore our problems

%matplotlib widget


# Creating a WCS solution

WCS solutions (as explained in Lab2 Photometry) use astrometric methods to find the field of view and plate-scale by using sets of stars. Below are a few methods on how to do this.

### Software

One common method is to use index files/software. The most common software is from https://astrometry.net/doc/readme.html. This allows you to use a terminal command to plate-solve (or python through a terminal). However, there are a few things to note:
1. This can only be installed with Linux or Macosx, Windows would have to use a virtual machine.
2. The index files to install take up to 150 GB of hard drive space - so typically they are not worth it for personal devices unless you build/reduce plenty of files.

The command to run this looks like: 
    "solve-field --no-plots --scale-units arcminwidth --scale-low {lower bound} --scale-high {upper bound} {filename}"
where the objects within the "{}" are to be input. If you do not know the field scale you can use a slower version of the command: "solve-field --no-plots {filename}". This will create a series of files, where the ".new" are the new files with the wcs header.


### Online

This will be the recommended method for most users. Using the same software as above the website: https://nova.astrometry.net/upload uses an API to connect to the index files, meaning that no install is required. All one has to do is upload the reduced file and wait. While this is more time consuming, it will work across all devices and does not require a lot of files.

#### !!! We will use this method for our reduction in later labs !!!

### Python

Below is a method to plate solve using the same software but using a python environment and your own API. This method will charge you for each plate-solve which makes it largely inefficient for large image reductions when software installation will be faster, can be multiprocessed, and free.


In [ ]:

from astroquery.astrometry_net import AstrometryNet

def plate_solve_api(file, ra=None, dec=None, radius=0.5, wcs_folder='wcs/'):
    '''
    This function uses the Astrometry.net API to plate solve an image. 
    This version will attempt to plate solve the image up to 12 times. 
    This version is a modified version that Ryan Ridden originally wrote 
    for Pouakai and Zachary Lane modified/adapted for FLI Pouakai and 
    the Kepler/K2 transient reduction pipeline FEVAR.
    '''
    
    ast = AstrometryNet()
    ast.api_key = # Your API key here, I'm not giving you mine
    
    header = fits.open(file)[0].header
    data = fits.open(file)[0].data
    
    filename = file.split('/')[-1]

    attempts_allowed = 12

    attempt = 0
    solved = False
    while (attempt < attempts_allowed) & (not solved):
        try:
            if (ra is not None) & (dec is not None):
                wcs_header = ast.solve_from_image(file, solve_timeout=120, 
                                                scale_units = 'arcsecperpix', scale_type='ul', 
                                                scale_upper = 5, scale_lower = 3, 
                                                crpix_center=True, center_ra = ra, center_dec = dec,
                                                radius = radius, parity = 0)

            else:
                wcs_header = ast.solve_from_image(file, solve_timeout=300, 
                                                scale_units = 'arcsecperpix', scale_type='ul', 
                                                scale_upper = 5, scale_lower = 3, 
                                                crpix_center=True)
            del wcs_header['COMMENT']
            del wcs_header['HISTORY']
            solved = True
        except:
            attempt += 1
    if (attempt > attempts_allowed) | (solved == False):
        print(f'Unable to plate solve: {file}')
        return None
    else:
        wcs = WCS(wcs_header, relax=True)
            
        # self.header['MJD'] = self.mjd

        wcs_header = header + wcs.to_header()
        wcs_file = wcs_folder + filename
        
        primary_hdu = fits.PrimaryHDU(data, header=wcs_header)

        hdul = fits.HDUList([primary_hdu])
        hdul.writeto(wcs_file, overwrite=True)

# World Coordinate System Use

Some astronomical files are 'plate-solved', e.g. astrometric methods have been used to determine where in the sky the object is. This can be done by multiple methods, and is covered by the lectures.

If you are wanting more information on how these are done or how you can apply these, I would recommend talking to a photometry data-reduction expert.

These coordinates makes some analysis far more simple

## Task 1

Below is a code to show how WCS can be used to track a target. Within this image is the target star "TYC 8555-338-1".

- Change the path to the Macdiarmid Server ASTR211/2024_Lab_Files/Phot_Skills_Lab/
- Use SIMBAD or another method to find the Right Ascension (RA) and Declination (Dec) of the target
- Convert these coordinates to degrees and input them in the following code
- If this is done correctly, you should be able to find the star and the coordinates due to the "all_world2pix" function in astropy

In [ ]:
folder = '/Users/zgl12/Research/ASTR211/Tut_Files/'

ra = 112.02092060702
dec = -54.680893145519995

folder = '/Users/zgl12/Research/ASTR211/Tut_Files/Adv_Skills/'
filename = 'TYC8555_338_1-0057_g_wcs.fits.gz'

hdu = fits.open(folder + filename)
header = hdu[0].header
data = hdu[0].data
hdu.close()

wcs = WCS(header)

x, y = wcs.all_world2pix(ra, dec, 0) # You can also use wcs.all_pix2world(x, y, 0) to convert from pixel to world coordinates

mjd = header['JD'] - 2400000.5

plt.figure()
plt.scatter(x, y, c = 'r', s = 10)
plt.imshow(data, vmin = np.nanpercentile(data, 5), vmax = np.nanpercentile(data, 95), cmap = 'gray')
plt.show()

plt.figure()
plt.scatter(x, y, c = 'r', s = 10)
plt.imshow(data, vmin = np.nanpercentile(data, 5), vmax = np.nanpercentile(data, 95), cmap = 'gray')
plt.xlim(x - 50, x + 50)
plt.ylim(y - 50, y + 50)
plt.show()


When we print the WCS object (you can treat it like a dictionary) in the code snippet below we see the information contained:

- Number of WCS axes (number of dimensions)
- CTYPE: The projection mapping. Often this will be a tangent plane method with a Simple Imaging Polynomial (SIP) distortion
- CRVAL: The RA and Dec in degrees at a certain point on the detector
- CRPIX: The pixel location the CRVAL corresponds to
- CD{i}_{j}: The matrix components for the image rotation and scale. NOTE: older WCS may use a PC matrix and a separate scaling factor.
- NAXIS: Image size

In [ ]:
wcs

You can also use "astropy.coordinates.angular_separation" to find the angular distance between two sets of angular coordinates.

# Aligning Images

Aligning images can be very useful for difference imaging or for colour imaging. Below we will talk about some methods for alignment.

## Aligning with WCS

You can use the WCS to align multiple images where the kernel cannot be convolved to match by a simple pixel shift. Pixel shifts are typically limited to integer amounts and cannot handle large amounts of rotation. WCS alignment uses spherical polygon coordinate projection to match the image coordinates - this is independent of the spatial resolution of the telescope (meaning different telescopes can be stacked). Solving this exactly can be computationally slow (~15-30s per image), so often interpolation strategies are employed.

Exactly solving the method will conserve the flux of the object, but not necessarily the PSF shape. Interpolation methods, while faster will not preserve flux exactly, nor the PSF shape.

Below is a method for installing and reprojecting WCS.

### Installation

Linux and Macosx: in a terminal type the following "pip install --user reproject" (or conda install with "conda install -c conda-forge reproject").

Windows: In a Windows or Anaconda powershell prompt type the following "pip install --user reproject" (or conda install with "conda install -c conda-forge reproject").

## Task 2

Below is a code to show how WCS can be used to align images.

- Align the two images "TYC8555_338_1-0057_g_wcs.fits.gz" and "TYC8555_338_1-0059_g_wcs.fits.gz" with each other using the below code and plot them

In [ ]:

from reproject import reproject_interp, reproject_exact

aligned_data, _ = reproject_exact(folder + 'TYC8555_338_1-0058_g_wcs.fits.gz', wcs)


In [ ]:
plt.figure()
plt.scatter(x, y, c = 'r', s = 10)
plt.imshow(data, vmin = np.nanpercentile(data, 5), vmax = np.nanpercentile(data, 95), cmap = 'gray')
plt.title('Image 1')
plt.show()

plt.figure()
plt.scatter(x, y, c = 'r', s = 10)
plt.imshow(aligned_data, vmin = np.nanpercentile(aligned_data, 5), vmax = np.nanpercentile(aligned_data, 95), cmap = 'gray')
plt.title('Image 2')
plt.show()

diff_imaged_data = data - aligned_data

plt.figure()
plt.scatter(x, y, c = 'r', s = 10)
plt.imshow(diff_imaged_data, vmin = np.nanpercentile(diff_imaged_data, 5), vmax = np.nanpercentile(diff_imaged_data, 95), cmap = 'gray')
plt.title('Differenced Image')
plt.show()

## Aligning with Cross-Correllation

You can use the cross-correllation to align multiple images by convolving the kernel to find an integer pixel shift offset. This does preserve PSF flux and shape. Cross-correllation works very well for star fields, but large extended objects (e.g. Tarantula Nebula) become difficult. It also suffers for undersampled PSFs.

For a static target, often it is easiest to plate-solve one image and then use cross-correllation to align the other images. That way you can find your target using the WCS and use the same pixel coordinates throughout all of your images without plate-solving every image.

Below is a code for aligning two images using cross-correllation.

In [ ]:

def register_images(target_image, reference_image, smoothing_kernel_size=3):
    """
    Register (align) the reference_image to the target_image using FFT-based cross-correlation.
    
    Parameters:
        target_image (2D np.ndarray): The image to align to (template).
        reference_image (2D np.ndarray): The image to be shifted (science frame).
        smoothing_kernel_size (int): Size of the box kernel to smooth the cross-correlation surface.
        
    Returns:
        aligned_image (2D np.ndarray): The shifted version of the target image aligned with the reference.
                                       Has the same shape as target_image.
        (x_shift, y_shift) (tuple): Pixel shifts in x (rows) and y (columns).
    """
    
    ref_centered = reference_image - np.median(reference_image) # Subtract the median to remove background and enhance matching
    tgt_centered = target_image - np.median(target_image)

    cross_corr = fftconvolve(ref_centered, tgt_centered[::-1, ::-1], mode='full') # Compute normalised cross-correlation via FFT

    
    kernel = np.ones((smoothing_kernel_size, smoothing_kernel_size)) # Optional smoothing to suppress noise in correlation peak
    cross_corr_smoothed = convolve2d(cross_corr, kernel, mode='same')

    max_idx = np.unravel_index(np.argmax(cross_corr_smoothed), cross_corr_smoothed.shape) # Find peak in cross-correlation to get shift
    peak_y, peak_x = max_idx

    x_shift = peak_y - reference_image.shape[0] + 1 # Calculate pixel shift from reference to target
    y_shift = peak_x - reference_image.shape[1] + 1

    print(f"Calculated shifts -> x: {x_shift} pixels, y: {y_shift} pixels")

    ref_y_start = max(0, x_shift) # Compute overlapping region after shift
    ref_y_end   = min(reference_image.shape[0], reference_image.shape[0] + x_shift)
    ref_x_start = max(0, y_shift)
    ref_x_end   = min(reference_image.shape[1], reference_image.shape[1] + y_shift)

    tgt_y_start = max(0, -x_shift)
    tgt_y_end   = min(target_image.shape[0], target_image.shape[0] - x_shift)
    tgt_x_start = max(0, -y_shift)
    tgt_x_end   = min(target_image.shape[1], target_image.shape[1] - y_shift)

    
    aligned_image = np.zeros_like(target_image) # Create an aligned image (zeros where there's no overlap)
    aligned_image[ref_y_start:ref_y_end, ref_x_start:ref_x_end] = target_image[tgt_y_start:tgt_y_end, tgt_x_start:tgt_x_end]

    return aligned_image


## Task 3

Use the above cross-correllation function to align images. 

- Align the two images "TYC8555_338_1-0057_g_wcs.fits.gz" and "TYC8555_338_1-0059_g_wcs.fits.gz" with each other using the above code and plot them
- What is the difference between the two methods? 

HINT: use a simplistic Difference Imaging to analyse this. This will not be perfect as we have not created/modelled the effective Point-Spread Function (ePSF) for both fields and convolved. But for a temporary imperfect system this will be sufficient.

- How does it affect the PSF shape/profile?

In [ ]:
hdu = fits.open(folder + 'TYC8555_338_1-0058_g_wcs.fits.gz')
header = hdu[0].header
new_data = hdu[0].data
hdu.close()

new_wcs = WCS(header)

aligned_new_image = register_images(new_data, data, smoothing_kernel_size=3)



In [ ]:
plt.figure()
plt.scatter(x, y, c = 'r', s = 10)
plt.imshow(data, vmin = np.nanpercentile(data, 5), vmax = np.nanpercentile(data, 95), cmap = 'gray')
plt.title('Image 1')
plt.show()

plt.figure()
plt.scatter(x, y, c = 'r', s = 10)
plt.imshow(aligned_new_image, vmin = np.nanpercentile(aligned_new_image, 5), vmax = np.nanpercentile(aligned_new_image, 95), cmap = 'gray')
plt.title('Image 2')
plt.show()

diff_imaged_data = data - aligned_new_image

plt.figure()
plt.scatter(x, y, c = 'r', s = 10)
plt.imshow(diff_imaged_data, vmin = np.nanpercentile(diff_imaged_data, 5), vmax = np.nanpercentile(diff_imaged_data, 95), cmap = 'gray')
plt.title('Differenced Image')
plt.show()

In [ ]:
# python3 client.py -u /Users/zgl12/Research/ASTR211/Comet/file_1.fit -k csffdfuichpbiata --scale-lower 24 --scale-lower 27 --scale-units arcminwidth --wcs /Users/zgl12/file1.wcs